In [20]:
import boto3
import sys
import json
import os
from boto3.dynamodb.conditions import Key
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from anthropic  import Anthropic
from langchain_community.embeddings import BedrockEmbeddings

from utils import download_folder_s3
sys.path.append("../../..")


from get_param import get_amplify_data, get_string_param

tmp_path                    = "./tmp"
TABLE_NAME, BUCKET_NAME = get_amplify_data()
PAGES_TABLE_NAME        = get_string_param("/table/pages")
pages_table             = boto3.resource('dynamodb').Table(PAGES_TABLE_NAME)
docum_table             = boto3.resource('dynamodb').Table(TABLE_NAME)
s3_prefix               = "private"

bedrock_client              = boto3.client("bedrock-runtime")


antro                   = Anthropic()

with open("event.json") as f:
    event= json.load(f)
#event

API: puytqg6zmffolkkrdlkskx4sny
Table: LicitacionDoc-puytqg6zmffolkkrdlkskx4sny-devbranch
Bucket: documents60bfd-devbranch


In [15]:
path_parameters         = event.get("pathParameters")
document_id             = path_parameters.get("documentId")
document                = docum_table.get_item(Key={"id": document_id}).get("Item")

body_json               = event.get("body")
body                    = json.loads(body_json)if body_json else {}
query                   = body.get("query")
num_docs                = body.get("num_docs")
query                   = query if query else ""
num_docs                = num_docs if num_docs else 5


embedding_model         = document.get("embeddingModel")
embedding_dimensions    = int(document.get("embeddingDimensions")) if document.get("embeddingDimensions") else  1024
embedding_normalize     = bool(document.get("embeddingNormalize")) if document.get("embeddingNormalize") else True
vector_store_type       = document.get("vectorStoreType") or "faiss"
file_name               = document.get("vectorStoreLocation")
ownerId                 = document.get("ownerId")
model_kwargs            = dict(dimensions=int(embedding_dimensions), normalize=embedding_normalize)
kwargs_embedding        = dict(model_id=embedding_model, client=bedrock_client)

if "amazon.titan-embed-text-v2" in embedding_model:
    kwargs_embedding["model_kwargs"] = model_kwargs

embeddings              = BedrockEmbeddings(**kwargs_embedding)

In [16]:
query, num_docs, embedding_model, embedding_dimensions, embedding_normalize

('que se solicita?', 6, 'amazon.titan-embed-text-v2:0', 1024, True)

In [22]:
vector_location = f"{s3_prefix}/{ownerId}/{file_name}"
local_path = f"{tmp_path}/{file_name}"

if not os.path.exists(local_path):
    print("archivo faiss no existe")
    download_folder_s3(BUCKET_NAME, vector_location, local_path)

archivo faiss no existe
archivo descargado


In [24]:
db                      = FAISS.load_local(local_path, embeddings, allow_dangerous_deserialization=True)
retriever               = db.as_retriever(search_kwargs = {'k':num_docs}, search_type = "mmr")
docs                    = retriever.invoke(query)
print (docs)

[Document(metadata={'source': 'Banco Central.pdf', 'page': 2}, page_content='# V. Consideraciones\n\n- Se hace presente que este proceso de Consultas al Mercado no constituye una Cotización, por lo que los Proveedores deben abstenerse de enviar propuestas económicas.\n- Esta solicitud se efectúa sólo para efectos informativos y no compromete en ningún caso al Banco Central de Chile para adquirir o contratar los Productos y/o Servicios individualizados.\n- En caso de que el Banco Central de Chile requiera la adquisición o contratación de estos Productos y/o Servicios, se iniciará un Proceso de Cotización o Licitación a través de su Departamento de Adquisiciones.\n\n# VI. Cuestionario'), Document(metadata={'source': 'Banco Central.pdf', 'page': 3}, page_content='3\n\n1.9\n\n¿La Empresa puede prestar el Servicio en 60 días corridos?\n\n2. En cuanto al Servicio:\n\n2.1\n\n¿Cuáles son las principales funcionalidades que ofrece la solución, y cuáles de ellos ha \nimplementado?\n\n2.2\n\n¿Cuá